# EDA
- This notebook conducts data analysis of the World Development Indicators Dataset
- The main variables of interest are GDP, consumption, investment, exports, imports, population, and the birth rate

In [1]:
%load_ext sql
%sql postgresql:///wdi.db

Traceback (most recent call last):
  File "/Users/mitch/envs/main/lib/python3.9/site-packages/sqlalchemy/engine/base.py", line 145, in __init__
    self._dbapi_connection = engine.raw_connection()
  File "/Users/mitch/envs/main/lib/python3.9/site-packages/sqlalchemy/engine/base.py", line 3293, in raw_connection
    return self.pool.connect()
  File "/Users/mitch/envs/main/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 452, in connect
    return _ConnectionFairy._checkout(self)
  File "/Users/mitch/envs/main/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 1269, in _checkout
    fairy = _ConnectionRecord.checkout(pool)
  File "/Users/mitch/envs/main/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 716, in checkout
    rec = pool._do_get()
  File "/Users/mitch/envs/main/lib/python3.9/site-packages/sqlalchemy/pool/impl.py", line 170, in _do_get
    self._dec_overflow()
  File "/Users/mitch/envs/main/lib/python3.9/site-packages/sqlalchemy/util/langhelpers.py

In [2]:
%%sql
postgresql:///wdi

## EDA

In [3]:
%%sql
SELECT * FROM main
LIMIT 10

 * postgresql:///wdi
10 rows affected.


country_code,country,indicator,year,value,growth_rate,region,income_group
ABW,Aruba,Birth_Rate,1960,35.679,None,Latin America & Caribbean,High income
ABW,Aruba,Birth_Rate,1961,34.529,-3.22,Latin America & Caribbean,High income
ABW,Aruba,Birth_Rate,1962,33.32,-3.5,Latin America & Caribbean,High income
ABW,Aruba,Birth_Rate,1963,32.05,-3.81,Latin America & Caribbean,High income
ABW,Aruba,Birth_Rate,1964,30.737,-4.1,Latin America & Caribbean,High income
ABW,Aruba,Birth_Rate,1965,29.413,-4.31,Latin America & Caribbean,High income
ABW,Aruba,Birth_Rate,1966,28.121,-4.39,Latin America & Caribbean,High income
ABW,Aruba,Birth_Rate,1967,26.908,-4.31,Latin America & Caribbean,High income
ABW,Aruba,Birth_Rate,1968,25.817,-4.05,Latin America & Caribbean,High income
ABW,Aruba,Birth_Rate,1969,24.872,-3.66,Latin America & Caribbean,High income


In [4]:
%%sql
SELECT * FROM pop
LIMIT 10

 * postgresql:///wdi
10 rows affected.


country_code,year,population,population_growth,region,income_group
ABW,1960,54208,None,Latin America & Caribbean,High income
ABW,1961,55434,2.26,Latin America & Caribbean,High income
ABW,1962,56234,1.44,Latin America & Caribbean,High income
ABW,1963,56699,0.83,Latin America & Caribbean,High income
ABW,1964,57029,0.58,Latin America & Caribbean,High income
ABW,1965,57357,0.58,Latin America & Caribbean,High income
ABW,1966,57702,0.6,Latin America & Caribbean,High income
ABW,1967,58044,0.59,Latin America & Caribbean,High income
ABW,1968,58377,0.57,Latin America & Caribbean,High income
ABW,1969,58734,0.61,Latin America & Caribbean,High income


In [5]:
%%sql
SELECT DISTINCT(indicator) "Indicator"
FROM main
ORDER BY indicator

 * postgresql:///wdi
7 rows affected.


Indicator
Birth_Rate
Consumption
Exports
GDP
Imports
Investment
Population


In [6]:
%%sql
SELECT DISTINCT(income_group) "Income Group"
FROM main
ORDER BY income_group

 * postgresql:///wdi
4 rows affected.


Income Group
High income
Low income
Lower middle income
Upper middle income


In [7]:
%%sql
SELECT DISTINCT(region) "Region"
FROM main
ORDER BY region

 * postgresql:///wdi
7 rows affected.


Region
East Asia & Pacific
Europe & Central Asia
Latin America & Caribbean
Middle East & North Africa
North America
South Asia
Sub-Saharan Africa


## Groups: Region

In [8]:
%%sql
SELECT region, ROUND(AVG(value)::numeric, 0) "Average GDP"
FROM main
WHERE indicator = 'GDP'
AND year = 2020
GROUP BY region
ORDER BY AVG(value)

 * postgresql:///wdi
7 rows affected.


region,Average GDP
Sub-Saharan Africa,38779423826
Latin America & Caribbean,137858206484
Middle East & North Africa,168104657507
South Asia,404457509131
Europe & Central Asia,420846057137
East Asia & Pacific,776186598896
North America,6969434023656


## Population

### Most Populated Countries

In [9]:
%%sql
SELECT country, value population
FROM main
WHERE indicator = 'Population'
AND year = 2020
ORDER BY population DESC
LIMIT 10

 * postgresql:///wdi
10 rows affected.


country,population
China,1410929362.0
India,1380004385.0
United States,329484123.0
Indonesia,273523621.0
Pakistan,220892331.0
Brazil,212559409.0
Nigeria,206139587.0
Bangladesh,164689383.0
Russia,144104080.0
Mexico,128932753.0


### Least Populated Countries

In [10]:
%%sql
SELECT country, value population
FROM main
WHERE indicator = 'Population'
AND year = 2020
ORDER BY population ASC
LIMIT 10

 * postgresql:///wdi
10 rows affected.


country,population
Nauru,10834.0
Tuvalu,11792.0
Palau,18092.0
British Virgin Islands,30237.0
Gibraltar,33691.0
San Marino,33938.0
Liechtenstein,38137.0
St. Martin (French part),38659.0
Turks and Caicos Islands,38718.0
Monaco,39244.0


### Fastest Growing

In [11]:
%%sql
SELECT country, region, growth_rate population_growth
FROM main
WHERE indicator = 'Population'
AND year = 2020
ORDER BY growth_rate DESC
LIMIT 10

 * postgresql:///wdi
10 rows affected.


country,region,population_growth
Malta,Middle East & North Africa,4.21
Niger,Sub-Saharan Africa,3.84
Bahrain,Middle East & North Africa,3.68
Equatorial Guinea,Sub-Saharan Africa,3.47
Uganda,Sub-Saharan Africa,3.32
Angola,Sub-Saharan Africa,3.27
Dem. Rep. Congo,Sub-Saharan Africa,3.19
Burundi,Sub-Saharan Africa,3.12
Mali,Sub-Saharan Africa,3.02
Chad,Sub-Saharan Africa,3.0


### Slowest Growing

In [12]:
%%sql
SELECT country, region, growth_rate population_growth
FROM main
WHERE indicator = 'Population'
AND year = 2020
ORDER BY growth_rate ASC
LIMIT 10

 * postgresql:///wdi
10 rows affected.


country,region,population_growth
Moldova,Europe & Central Asia,-1.67
Curaçao,Latin America & Caribbean,-1.54
Kosovo,Europe & Central Asia,-0.75
Latvia,Europe & Central Asia,-0.64
Bosnia and Herzegovina,Europe & Central Asia,-0.61
Bulgaria,Europe & Central Asia,-0.6
Albania,Europe & Central Asia,-0.58
Ukraine,Europe & Central Asia,-0.57
Serbia,Europe & Central Asia,-0.53
Romania,Europe & Central Asia,-0.44


## Birth Rate

### Highest Birth Rate
- The countries with the highest birth rate are Niger, Chad, and Somolia
- The majority of high birth rate countries are all in Sub-Saharan Africa

In [13]:
%%sql
SELECT country, region, value "Birth Rate"
FROM main
WHERE year = 2019
AND indicator = 'Birth_Rate'
ORDER BY value DESC
LIMIT 10

 * postgresql:///wdi
10 rows affected.


country,region,Birth Rate
Niger,Sub-Saharan Africa,45.637
Chad,Sub-Saharan Africa,41.665
Somalia,Sub-Saharan Africa,41.585
Mali,Sub-Saharan Africa,41.035
Dem. Rep. Congo,Sub-Saharan Africa,40.639
Angola,Sub-Saharan Africa,40.232
Burundi,Sub-Saharan Africa,38.377
The Gambia,Sub-Saharan Africa,38.073
Burkina Faso,Sub-Saharan Africa,37.453
Nigeria,Sub-Saharan Africa,37.446


### Lowest Birth Rate
- The countries with the lowest birth rates are Korea, Puerto Rico, San Marino, and Japan
- The majority of low birth rate countries are in Europe/Central Asia and East Asia/Pacific

In [14]:
%%sql
SELECT country, region, value "Birth Rate"
FROM main
WHERE year = 2019
AND indicator = 'Birth_Rate'
ORDER BY value ASC
LIMIT 10

 * postgresql:///wdi
10 rows affected.


country,region,Birth Rate
Korea,East Asia & Pacific,5.9
Puerto Rico,Latin America & Caribbean,6.4
San Marino,Europe & Central Asia,6.7
Japan,East Asia & Pacific,7.0
Andorra,Europe & Central Asia,7.0
Italy,Europe & Central Asia,7.0
"Hong Kong SAR, China",East Asia & Pacific,7.0
Spain,Europe & Central Asia,7.6
Greece,Europe & Central Asia,7.8
Bosnia and Herzegovina,Europe & Central Asia,7.938


## Groups: Income

### Income Share
- What percent of world income does each income group take up?
- The high income group takes up more than half of world income, at 61%.
- The upper middle income group takes another 29%.
- The low income group only earns 0.5% of world income.

In [15]:
%%sql
SELECT income_group, 
ROUND(100*(SUM(value) / (SELECT SUM(value) FROM main WHERE indicator = 'GDP' AND year = 2020))::numeric, 2) income_share
FROM main
WHERE indicator = 'GDP'
AND year = 2020
GROUP BY income_group
ORDER BY SUM(value) DESC

 * postgresql:///wdi
4 rows affected.


income_group,income_share
High income,61.21
Upper middle income,28.93
Lower middle income,9.34
Low income,0.52


### Population Share

In [16]:
%%sql
SELECT income_group, 
ROUND(100*(SUM(value) / (SELECT SUM(value) FROM main WHERE indicator = 'Population' AND year = 2020))::numeric, 2) "Population Share"
FROM main
WHERE indicator = 'Population'
AND year = 2020
GROUP BY income_group
ORDER BY SUM(value) DESC

 * postgresql:///wdi
4 rows affected.


income_group,Population Share
Lower middle income,43.07
Upper middle income,32.34
High income,15.77
Low income,8.82


## GDP Per Capita

In [17]:
%%sql
DROP TABLE IF EXISTS gdp_table

 * postgresql:///wdi
Done.


[]

- GDP per Capita is the total GDP of the economy divided by the population
- It serves as a broad measure of the average 'income' in each country
- This query computes GDP per Capita for each country-year instance

In [18]:
%%sql
SELECT country,
main.region,
main.income_group,
main.year,
ROUND((value / population)::numeric, 0) "GDP per Capita"
INTO gdp_table
FROM main
LEFT JOIN pop
ON main.country_code = pop.country_code
AND main.year = pop.year
WHERE main.indicator = 'GDP'

 * postgresql:///wdi
9598 rows affected.


[]

In [19]:
%%sql
SELECT * 
FROM gdp_table
LIMIT 10

 * postgresql:///wdi
10 rows affected.


country,region,income_group,year,GDP per Capita
Aruba,Latin America & Caribbean,High income,1986,17231
Aruba,Latin America & Caribbean,High income,1987,20263
Aruba,Latin America & Caribbean,High income,1988,24343
Aruba,Latin America & Caribbean,High income,1989,27313
Aruba,Latin America & Caribbean,High income,1990,27884
Aruba,Latin America & Caribbean,High income,1991,28954
Aruba,Latin America & Caribbean,High income,1992,29032
Aruba,Latin America & Caribbean,High income,1993,29325
Aruba,Latin America & Caribbean,High income,1994,29989
Aruba,Latin America & Caribbean,High income,1995,29367


### Richest Countries
- Which countries have the highest GDP per capita
- We can see Monaco, Luxembourg, and Bermuda have the highest GDP per capita

In [20]:
%%sql
SELECT * FROM gdp_table
WHERE year = 2020
ORDER BY "GDP per Capita" DESC
LIMIT 10

 * postgresql:///wdi
10 rows affected.


country,region,income_group,year,GDP per Capita
Monaco,Europe & Central Asia,High income,2020,159222
Luxembourg,Europe & Central Asia,High income,2020,104529
Bermuda,North America,High income,2020,100412
Switzerland,Europe & Central Asia,High income,2020,85682
Ireland,Europe & Central Asia,High income,2020,78558
Cayman Islands,Latin America & Caribbean,High income,2020,76999
Norway,Europe & Central Asia,High income,2020,75017
United States,North America,High income,2020,58560
Singapore,East Asia & Pacific,High income,2020,58057
Australia,East Asia & Pacific,High income,2020,58044


### Poorest Countries
- Burundi, Malawi, and Central African Republic have the lowest GDP per capita.
- The majority of the lowest income countries are in Sub-Saharan Africa

In [21]:
%%sql
SELECT * FROM gdp_table
WHERE year = 2020
ORDER BY "GDP per Capita" ASC
LIMIT 10

 * postgresql:///wdi
10 rows affected.


country,region,income_group,year,GDP per Capita
Burundi,Sub-Saharan Africa,Low income,2020,271
Malawi,Sub-Saharan Africa,Low income,2020,394
Central African Republic,Sub-Saharan Africa,Low income,2020,414
Madagascar,Sub-Saharan Africa,Low income,2020,442
Somalia,Sub-Saharan Africa,Low income,2020,445
Dem. Rep. Congo,Sub-Saharan Africa,Low income,2020,505
Niger,Sub-Saharan Africa,Low income,2020,523
Afghanistan,South Asia,Low income,2020,530
Mozambique,Sub-Saharan Africa,Low income,2020,575
Liberia,Sub-Saharan Africa,Low income,2020,616


### Average GDP Per Capita by Income Group

In [22]:
%%sql
SELECT income_group, 
ROUND(AVG("GDP per Capita")::numeric, 0) "Average GDP per Capita"
FROM gdp_table
WHERE year = 2020
GROUP BY income_group
ORDER BY AVG("GDP per Capita") DESC

 * postgresql:///wdi
4 rows affected.


income_group,Average GDP per Capita
High income,36272
Upper middle income,6812
Lower middle income,2449
Low income,668


### Average GDP Per Capita by Region
- Here we can see North America and Europe/Central Asia have the highest average GDP per capita.
- South Asia and Sub-Saharan Africa have the lowest.

In [23]:
%%sql
SELECT region, 
ROUND(AVG("GDP per Capita")::numeric , 0) "Average GDP per Capita"
FROM gdp_table
WHERE year = 2020
GROUP BY region
ORDER BY AVG("GDP per Capita") DESC

 * postgresql:///wdi
7 rows affected.


region,Average GDP per Capita
North America,67089
Europe & Central Asia,27439
Middle East & North Africa,14588
East Asia & Pacific,14449
Latin America & Caribbean,11664
South Asia,2507
Sub-Saharan Africa,2155


## Group: Region

### Population Share
- East Asia/Pacific take up the highest population share, whereas North America takes the lowest

In [24]:
%%sql
SELECT region "Region",
 ROUND(100*(SUM(value) / (SELECT SUM(value) FROM main WHERE indicator = 'Population' AND year = 2020))::numeric, 2) "Population Share (%)"
FROM main
WHERE indicator = 'Population'
AND year = 2020
GROUP BY region
ORDER BY SUM(value) DESC

 * postgresql:///wdi
7 rows affected.


Region,Population Share (%)
East Asia & Pacific,30.33
South Asia,24.10
Sub-Saharan Africa,14.70
Europe & Central Asia,11.98
Latin America & Caribbean,8.10
Middle East & North Africa,6.03
North America,4.77


### Income Share

In [25]:
%%sql
SELECT region "Region",
 ROUND(100*(SUM(value) / (SELECT SUM(value) FROM main WHERE indicator = 'GDP' AND year = 2020))::numeric, 2) "Income Share (%)"
FROM main
WHERE indicator = 'GDP'
AND year = 2020
GROUP BY region
ORDER BY SUM(value) DESC

 * postgresql:///wdi
7 rows affected.


Region,Income Share (%)
East Asia & Pacific,31.72
Europe & Central Asia,26.06
North America,25.90
Latin America & Caribbean,6.15
South Asia,4.01
Middle East & North Africa,3.96
Sub-Saharan Africa,2.21


# Covid 19 Crisis

- This section examines the covid-19 crisis
- Across countries, the crisis produced an average decline in GDP of 5%

In [26]:
%%sql
SELECT year, 
ROUND(AVG(growth_rate)::numeric, 2) "Average GDP Growth"
FROM main
WHERE indicator = 'GDP'
AND year BETWEEN 2015 AND 2020
GROUP BY year

 * postgresql:///wdi
6 rows affected.


year,Average GDP Growth
2017,3.17
2015,2.66
2016,3.14
2018,3.06
2019,2.81
2020,-5.00


- How did each component of national accounts change?
- We can access this by grouping on the indicator and computing the average over all countries
- Exports faced the strongest downturn, at 11.5%
- GDP faced a downturn of 5%, whereas consumption displayed a smaller downturn of 2.68%

In [27]:
%%sql
SELECT indicator, 
ROUND(AVG(growth_rate)::numeric, 2) "Percentage Change"
FROM main
WHERE year = 2020
GROUP BY indicator
ORDER BY AVG(growth_rate)

 * postgresql:///wdi
6 rows affected.


indicator,Percentage Change
Exports,-11.48
Imports,-9.65
Investment,-7.82
GDP,-5.00
Consumption,-2.68
Population,1.16


## Covid 19: Which Countries fared the Worst?

- Which countries fared worst during the crisis?
- We can Macao SAR, the Maldives, and Libya fared the worst
- Countries with the worst downturns were typically in Latin America and the Caribbean 

In [28]:
%%sql
SELECT country, region, income_group,
growth_rate downturn
FROM main
WHERE indicator = 'GDP'
AND year = 2020
ORDER BY growth_rate
LIMIT 10

 * postgresql:///wdi
10 rows affected.


country,region,income_group,downturn
"Macao SAR, China",East Asia & Pacific,High income,-54.01
Maldives,South Asia,Upper middle income,-33.5
Libya,Middle East & North Africa,Upper middle income,-31.3
Turks and Caicos Islands,Latin America & Caribbean,High income,-26.78
Lebanon,Middle East & North Africa,Lower middle income,-21.46
St. Lucia,Latin America & Caribbean,Upper middle income,-20.37
Antigua and Barbuda,Latin America & Caribbean,High income,-20.19
Barbados,Latin America & Caribbean,High income,-18.98
Curaçao,Latin America & Caribbean,High income,-18.4
Panama,Latin America & Caribbean,High income,-17.94


### Larger Countries
- The countries with the largest downturns are all quite small
- What about larger countries?
- To study this, I restrict that each countries population share is more than 0.5% of the world population
- Using this restriction, we can see that Iraq, Spain, and Argentina fared the worst

In [29]:
%%sql
WITH mod AS (
    SELECT main.country, main.growth_rate, main.income_group, main.region, 
    ROUND((100*pop.population / (SELECT SUM(population) FROM pop WHERE year = 2020)), 2)::numeric population_share
    FROM 
    main LEFT JOIN  pop
    ON main.country_code = pop.country_code
    AND main.year = pop.year
    WHERE main.year = 2020
    AND indicator = 'GDP'
)
SELECT * FROM mod
WHERE population_share >= 0.5
ORDER BY growth_rate ASC
LIMIT 20

 * postgresql:///wdi
20 rows affected.


country,growth_rate,income_group,region,population_share
Iraq,-15.67,Upper middle income,Middle East & North Africa,0.52
Spain,-10.82,High income,Europe & Central Asia,0.61
Argentina,-9.9,Upper middle income,Latin America & Caribbean,0.59
Philippines,-9.57,Lower middle income,East Asia & Pacific,1.42
United Kingdom,-9.4,High income,Europe & Central Asia,0.87
Italy,-8.94,High income,Europe & Central Asia,0.77
Mexico,-8.31,Upper middle income,Latin America & Caribbean,1.67
France,-7.86,High income,Europe & Central Asia,0.87
India,-7.25,Lower middle income,South Asia,17.91
Colombia,-6.8,Upper middle income,Latin America & Caribbean,0.66


In [52]:
%%sql
-- computing the moving average
SELECT country_code, year, 
ROUND(AVG(growth_rate) OVER(
    PARTITION BY country_code ORDER BY year ASC ROWS BETWEEN 11 PRECEDING AND 1 PRECEDING
    )::numeric, 2) moving_average
FROM main
WHERE 
indicator = 'GDP'
AND country_code = 'USA'
ORDER BY year DESC
LIMIT 20

 * postgresql:///wdi
20 rows affected.


country_code,year,moving_average
USA,2020,1.86
USA,2019,1.65
USA,2018,1.55
USA,2017,1.60
USA,2016,1.76
USA,2015,1.83
USA,2014,1.86
USA,2013,1.85
USA,2012,1.73
USA,2011,1.97


# Global Financial Crisis

- How did national accounts change during the global financial crisis?
- We observe an average decline in GDP of only 0.02%.
- Imports, exports, and investment display larger declines.

In [223]:
%%sql
SELECT indicator, 
ROUND(AVG(growth_rate)::numeric, 2) "Percentage Change"
FROM main
WHERE year = 2009
GROUP BY indicator
ORDER BY AVG(growth_rate)

 * postgresql:///wdi
7 rows affected.


indicator,Percentage Change
Imports,-6.95
Exports,-5.70
Investment,-3.77
Birth_Rate,-0.82
GDP,-0.02
Population,1.49
Consumption,2.06


### Global Financial Crisis: High Income Countries
- Let's focus on high income countries.
- Here we can see the decline is more dramatic relative to the average.
- Why? The global financial crisis was a financial crisis, so it disproportionately impacted high income countries.
- GDP declines by 3.6% on average.
- Investment displayed a sharped decline of 12.7%.

In [224]:
%%sql
SELECT indicator, 
ROUND(AVG(growth_rate)::numeric, 2) "Percentage Change"
FROM main
WHERE year = 2009
AND income_group = 'High income'
GROUP BY indicator
ORDER BY AVG(growth_rate)

 * postgresql:///wdi
7 rows affected.


indicator,Percentage Change
Investment,-12.66
Imports,-12.63
Exports,-8.72
GDP,-3.60
Consumption,-1.00
Birth_Rate,-0.85
Population,1.27


# Long Run Changes
- This section studies long run changes in GDP per capita

## Least Improved
- Which countries faced the largest decline in GDP per capita from 2000 to 2019?
- The countries with the largest decline are Guinea, Syria, and Libya

In [225]:
%%sql
WITH focus AS (
    SELECT *, 
    LAG("GDP per Capita", 1) OVER(PARTITION BY country ORDER BY year ASC) LGDP
    FROM gdp_table
    WHERE year = 2010
    OR year = 2019
)
SELECT country, region,
ROUND((100*("GDP per Capita" - LGDP )/ LGDP )::numeric, 2) change
FROM focus
WHERE year = 2019
ORDER BY (100*("GDP per Capita" - LGDP )/ LGDP ) ASC
LIMIT 10

 * postgresql:///wdi
10 rows affected.


country,region,change
Equatorial Guinea,Sub-Saharan Africa,-46.72
Syrian Arab Republic,Middle East & North Africa,-40.14
Libya,Middle East & North Africa,-32.68
Sudan,Sub-Saharan Africa,-30.05
Congo,Sub-Saharan Africa,-26.01
Lebanon,Middle East & North Africa,-25.32
Virgin Islands,Latin America & Caribbean,-23.75
Central African Republic,Sub-Saharan Africa,-21.43
Oman,Middle East & North Africa,-19.12
San Marino,Europe & Central Asia,-15.54


## Most Improved
- Which countries faced the largest increase in GDP per capita from 2000 to 2019?
- The countries with the largest increase are Nauru, Turkmenistan, and China
- The majority of countries are in East Asia/ Pacific

In [226]:
%%sql
WITH focus AS (
    SELECT *, 
    LAG("GDP per Capita", 1) OVER(PARTITION BY country ORDER BY year ASC) LGDP
    FROM gdp_table
    WHERE year = 2010
    OR year = 2019
)
SELECT country, region,
ROUND((100*("GDP per Capita" - LGDP )/ LGDP )::numeric, 2) change
FROM focus
WHERE year = 2019
AND LGDP IS NOT NULL
ORDER BY (100*("GDP per Capita" - LGDP )/ LGDP ) DESC
LIMIT 10

 * postgresql:///wdi
10 rows affected.


country,region,change
Nauru,East Asia & Pacific,102.23
Turkmenistan,Europe & Central Asia,80.33
China,East Asia & Pacific,79.83
Ethiopia,Sub-Saharan Africa,76.60
Myanmar,East Asia & Pacific,71.43
Mongolia,East Asia & Pacific,67.11
Bangladesh,South Asia,64.85
Ireland,Europe & Central Asia,63.59
Lao PDR,East Asia & Pacific,62.61
Cambodia,East Asia & Pacific,61.55
